## Do I need to clean my data?

The short answer is **always** assume your data needs to be cleaned. Data collection is rarely perfect, and even if the data is collected perfectly there are multiple points in the data pipeline where errors can be introduced. Data errors can be catastrophic for a myriad of reasons; modern analytic methods require data to confirm to certain standards for your analysis to be valid, and in some cases regression models will not run if your data is not properly cleaned. For the purposes of this demonstration I will be using the 2003 Stop, Question and Frisk dataset from the [NYPD](https://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page). I also recommend downloading the data dictionary from [here](https://data.cityofnewyork.us/api/views/ftxv-d5ix/files/9cecce6d-baca-4a60-be9d-31501b5639f0?download=true&filename=2016-sqf-file-spec.xlsx).

Because the data is not in stata format (\*.dta), we must import it into stata using the `import` command. In this case we will be using `import delimited` because the data is in text format with a delimiter. A delimiter separates values in your dataset. The most common type of delimiter is a comma, and files stored in this format usually end in .csv, which stands for comma separated values


In [1]:
import delimited /home/omari/Downloads/2003.csv, clear

(111 vars, 160,851 obs)


Before proceeding to clean your data, it is often useful to look at metadata. Metadata is simply data that describes your data, and is usually available for public use datasets. The most common forms of metadata you'll encounter are data dictionaries and codebooks. See a detailed description of codebooks [here](https://www.icpsr.umich.edu/icpsrweb/content/shared/ICPSR/faqs/what-is-a-codebook.html). Data dictonaries are much leaner than codebooks, and focus primarily on providing information about data formats.

In [2]:
/* This is a Stata comment. It allows us to write notes in our code that Stata will not execute.
If we do not specify it as a comment, Stata will attempt run our notes as code and will return a syntax error.
Multi line comments must be enclosed between /* ... */ */

// Alternatively, single line comments only require // and don't need to be enclosed. 

// describe lists all variables in our datasets and gives information about their storage type
describe 


Contains data
  obs:       160,851                          
 vars:           111                          
 size:    58,388,913                          
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
year            int     %8.0g                 
pct             int     %8.0g                 
ser_num         long    %12.0g                
datestop        long    %12.0g                
timestop        str5    %9s               

The two most frequent types of variables you'll encounter in Stata are character and numeric. Numeric variables can be stored as `int`, `byte`, `float`, `long` and `double`; character variables are stored as `str` (strings). The number at the end of `str` gives the width of the string, which is the maximum number of characters the string can hold. An important part of data cleaning is ensuring your variables are in the right format. 

In [3]:
/* The describe command from earlier can take variables as arguments. */
describe age


              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
age             str4    %9s                   


In [4]:
sum age


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
         age |          0


In [5]:
tab age


        age |      Freq.     Percent        Cum.
------------+-----------------------------------
            |        162        0.10        0.10
          0 |      6,349        3.95        4.05
          1 |        168        0.10        4.15
         10 |         42        0.03        4.18
        103 |          1        0.00        4.18
         11 |         61        0.04        4.22
        110 |          1        0.00        4.22
        113 |          1        0.00        4.22
        114 |          1        0.00        4.22
        115 |          1        0.00        4.22
         12 |        268        0.17        4.39
        120 |          5        0.00        4.39
        121 |          1        0.00        4.39
       1271 |          1        0.00        4.39
         13 |        811        0.50        4.89
        130 |          7        0.00        4.90
       1316 |          1        0.00        4.90
        132 |          1        0.00        4.90
        135 |      

Stata returns 0 observations when we try to calculate the mean and standard deviation of age using the `sum` (summarize) command, However, when we `tab` (tabulate) it shows us a range of values and more than 0 observations.  This is because age is stored as a string variable, and we are unable to perform arithmetic operations on string variables. In order for us to include age in our analysis, we must convert it to a numeric variable. As we saw earlier, there are multiple formats for numerical variables, so which format should we convert age to? `byte`, `int`, and `long` can only store integers (whole numbers), whereas `float` and `double` can store both integers and decimals. `float` and `double` consume more memory to store information, so it is not computationally efficient to store a whole numbers as a `float`. See this Stata [help file](https://www.stata.com/manuals13/ddatatypes.pdf) for more information. To determine what numeric format we should convert age to we must first examine the values of age.

We see that all the values of age are whole numbers, so we should not convert it to a `float` or `double`. There are also values of age that exceeds 100, so it cannot be stored as a `byte`. Logically, it makes sense to store it as an integer. We'll do so using the `destring` command.

In [8]:
destring age, gen(age_cleaned)

age: all characters numeric; age_cleaned generated as int
(162 missing values generated)


Byte can only store a maximum value of 100, and we can see from the frequency table that there are values of age that exceed 100. Because of this Stata choses to convert age to `int`, and our optional argument `gen()` creates a new variable (or column). Stata stores all datasets as 2-dimensional arrays; variables are stored as columns and observations are stored as rows. Alternatively, we could have specified `replace` in lieu of `gen()` and Stata would have overwritten our age column instead of creating a new one. This option is risky and should be used with caution. Another thing you may have noticed is that Stata created 162 missing values. This is the result of Stata converting an empty string to a numeric value. The first row of our frequency table shows 162 people with no values for age.

In [15]:
sum age_cleaned


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
 age_cleaned |    160,689    31.76657    225.3125          0       9999


In [7]:
sum age_cleaned, d


                         age_cleaned
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%           14              0
10%           16              0       Obs             160,689
25%           18              0       Sum of Wgt.     160,689

50%           23                      Mean           31.76657
                        Largest       Std. Dev.      225.3125
75%           32           9999
90%           43           9999       Variance       50765.73
95%           49           9999       Skewness       43.67801
99%           72           9999       Kurtosis       1930.646


We're now able to calculate the mean of age using our "cleaned" age variable. However our detailed summary (that's what the `, d` stands for) shows that we have rather unrealistic values of age, such as 9999. The most straightforward way to deal with unrealistic values of a variable is to replaced them as missing, thus removing the values from our analysis and preventing them from potentially skewing our results. While this may be the most straightforward way, it is often the least desirable way due to the loss of observations.  

In [9]:
replace age_cleaned = . if age_cleaned>100

(405 real changes made, 405 to missing)


In [10]:
sum age_cleaned, d


                         age_cleaned
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%           14              0
10%           16              0       Obs             160,284
25%           18              0       Sum of Wgt.     160,284

50%           23                      Mean           26.01252
                        Largest       Std. Dev.       12.9699
75%           32             99
90%           42             99       Variance       168.2183
95%           49             99       Skewness        1.52006
99%           64             99       Kurtosis       8.901667


We now see that the mean value of age is slightly lower, and the standard deviation is much smaller. Given the extremely small number of observations that are lost due to converting the extreme values to missing (405 out of 160,851), it may be sufficient to simply leave the values as missing. However if we were to lose more substantial amounts of observations by converting the extreme values to missing, it would be necessary to exlpore ways to impute those missing values with a more realistic number. Imputing the mean value is a popular choice. There are also statistical methods that can be used to impute missing values by way of predictive modeling. The latter is much more advanced than this tutorial aims to be, so we'll resort to the former.

In [11]:
return list


scalars:
                r(p99) =  64
                r(p95) =  49
                r(p90) =  42
                r(p75) =  32
                r(p50) =  23
                r(p25) =  18
                r(p10) =  16
                 r(p5) =  14
                 r(p1) =  0
                r(max) =  99
                r(min) =  0
                r(sum) =  4169391
           r(kurtosis) =  8.901666570994999
           r(skewness) =  1.520059538798137
                 r(sd) =  12.96990027108677
                r(Var) =  168.2183130419367
               r(mean) =  26.01252152429438
              r(sum_w) =  160284
                  r(N) =  160284


Stata stores the results of most of its computational commands in temporary scalars/vectors/matrices that can be viewed with `return list`. It is temporary because the next computational command will overwrite the current content of `return list`, and all content is lost when Stata is closed.

In [12]:
replace age_cleaned = r(mean) if age_cleaned==.

variable age_cleaned was int now float
(567 real changes made)


We' ve now replaced the 405 missing values we created in our previous step, as well as the 162 missing values that were already by imputing the mean value.

In [15]:
sum age_cleaned, d


                         age_cleaned
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%           14              0
10%           16              0       Obs             160,851
25%           18              0       Sum of Wgt.     160,851

50%           23                      Mean           26.01252
                        Largest       Std. Dev.      12.94702
75%           32             99
90%           42             99       Variance       167.6253
95%           48             99       Skewness       1.522746
99%           64             99       Kurtosis       8.933156


Because r(mean) was a float (had decimal places), it changed our age_cleaned from int to float. In order to return to integer, we can use the `recast` command which will truncate all floats and keep only the numbers to the left of the decimal. Note, truncation does not round up or round down. It only returns numbers to the left of the decimal.

In [17]:
tab age_cleaned


age_cleaned |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |      6,349        3.95        3.95
          1 |        168        0.10        4.05
          2 |         37        0.02        4.07
          3 |         23        0.01        4.09
          4 |         12        0.01        4.10
          5 |         83        0.05        4.15
          6 |         21        0.01        4.16
          7 |          5        0.00        4.16
          8 |          3        0.00        4.17
          9 |          5        0.00        4.17
         10 |         42        0.03        4.20
         11 |         61        0.04        4.23
         12 |        268        0.17        4.40
         13 |        811        0.50        4.90
         14 |      2,383        1.48        6.39
         15 |      4,893        3.04        9.43
         16 |      7,340        4.56       13.99
         17 |      9,063        5.63       19.62
         18 |     1

In [18]:
recast int age_cleaned

age_cleaned:  567 values would be changed; not changed


`recast` produces an error because Stata does not want to truncate your floats. You can override this with the optional argument `force`.

In [19]:
recast int age_cleaned, force

age_cleaned:  567 values changed


In [20]:
tab age_cleaned


age_cleaned |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |      6,349        3.95        3.95
          1 |        168        0.10        4.05
          2 |         37        0.02        4.07
          3 |         23        0.01        4.09
          4 |         12        0.01        4.10
          5 |         83        0.05        4.15
          6 |         21        0.01        4.16
          7 |          5        0.00        4.16
          8 |          3        0.00        4.17
          9 |          5        0.00        4.17
         10 |         42        0.03        4.20
         11 |         61        0.04        4.23
         12 |        268        0.17        4.40
         13 |        811        0.50        4.90
         14 |      2,383        1.48        6.39
         15 |      4,893        3.04        9.43
         16 |      7,340        4.56       13.99
         17 |      9,063        5.63       19.62
         18 |     1